In [1]:
import pandas as pd

In [2]:
df = pd.ExcelFile('backtest_transactions.xlsx')

In [3]:
df.sheet_names

['Sheet1']

In [4]:
df = df.parse(sheet_name='Sheet1')

In [5]:
india_vix = pd.read_csv('india_vix_updated_version.csv')
india_vix['full_date'] = pd.to_datetime(india_vix['Date'])


/tmp/ipykernel_11023/248907195.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  india_vix['full_date'] = pd.to_datetime(india_vix['Date'])


In [6]:
df.columns

Index(['Date', 'TradingSymbol', 'Qty', 'TransactionType', 'EntryTime',
       'EntryPrice', 'ExitTime', 'ExitPrice', 'StopLoss', 'Target',
       'TargetSLType', 'TradeCycle', 'OrderExitType', 'OrderStatus', 'PNL',
       'EntryDate', 'SquareOffDate'],
      dtype='object')

In [7]:
df['timestamp'] = pd.to_datetime(df['EntryTime'], format="%Y-%m-%dT%H:%M:%S%z")

In [8]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df['day_of_week'] = df['timestamp'].dt.day_name()
df['full_date'] = pd.to_datetime(df[['year', 'month', 'day']])


In [9]:
df = df[df["OrderStatus"]=="COMPLETED"]
manav = df[['year', 'month', 'day', 'hour', 'minute', 'day_of_week', 'PNL']]
# manav.to_csv("lokk.csv")

In [61]:
merged_df = india_vix.merge(manav, left_on="full_date", right_on="full_date", how="inner")

In [62]:
merged_df.columns

Index(['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Prev. Close',
       'Change', '% Change', 'open %change', 'low %change', 'high %change',
       'full_date', 'year', 'month', 'day', 'hour', 'minute', 'day_of_week',
       'PNL'],
      dtype='object')

In [63]:
merged_df = merged_df[['Date','PNL','% Change', 'open %change', 'low %change', 'high %change']]
merged_df.to_csv("manav.csv")

In [10]:
analyze = df.groupby(['year', 'month'])[['PNL']].sum()

In [11]:
analyze.to_csv("analysis_0.csv")

In [12]:
analyze = df.groupby(['year', 'month', 'hour'])[['PNL']].sum()

In [13]:
analyze.to_csv("analysis.csv")

In [10]:
analyze = df.groupby(['year', 'month', 'day', 'day_of_week'])[['PNL']].sum().reset_index()

In [11]:
analyze = analyze[(analyze['year'] >= 2024)]
analyze.to_csv("lokk2.csv", index=False)